# Wikipedia Notable Life Expectancies
# [Notebook  9: Data Cleaning Part 8](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_life_expect_data_clean8_thanak_2022_07_26.ipynb)
### Context

The
### Objective

The
### Data Dictionary
- Feature: Description

### Importing Libraries

In [1]:
# To structure code automatically
%load_ext nb_black

# To import/export sqlite databases
import sqlite3 as sql

# To save/open python objects in pickle file
import pickle

# To help with reading, cleaning, and manipulating data
import pandas as pd
import numpy as np
import re

# To define maximum number of columns to be displayed in a dataframe
pd.set_option("display.max_columns", None)
# To define the maximum number of rows to be displayed in a dataframe
pd.set_option("display.max_rows", 200)

# To supress warnings
# import warnings

# warnings.filterwarnings("ignore")

# To set some visualization attributes
pd.set_option("max_colwidth", 150)

# To play auditory cue when cell has executed, has warning, or has error and set chime theme
import chime

chime.theme("zelda")

<IPython.core.display.Javascript object>

## Data Overview

### Reading, Sampling, and Checking Data Shape

In [2]:
# Reading the dataset
conn = sql.connect("wp_life_expect_clean7.db")
data = pd.read_sql("SELECT * FROM wp_life_expect_clean7", conn)

# Making a working copy
df = data.copy()

# Checking the shape
print(f"There are {df.shape[0]} rows and {df.shape[1]} columns.")

# Checking first 2 rows of the data
df.head(2)

There are 98059 rows and 38 columns.


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
0,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21,1994,January,None,None,dancer,ballet designer and director,None,None,None,None,None,None,None,None,86.0,None,United Kingdom of Great Britain and Northern Ireland,None,None,3.091042,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12,1994,January,None,None,,writer,and academic,None,None,None,None,None,None,None,68.0,None,Ireland,None,None,2.564949,0,0,0,0,0,0,0,0,1,0,0,0,1


<IPython.core.display.Javascript object>

In [3]:
# Checking last 2 rows of the data
df.tail(2)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
98057,9,Aamir Liaquat Hussain,", 50, Pakistani journalist and politician, MNA .",https://en.wikipedia.org/wiki/Aamir_Liaquat_Hussain,99,2022,June,", since",None,,MNA,None,None,None,None,None,None,None,None,50.0,None,Pakistan,None,"2002 2007, since 2018",4.605170,0,0,0,0,0,1,0,0,1,0,0,0,2
98058,9,Zou Jing,", 86, Chinese engineer, member of the Chinese Academy of Engineering.",https://en.wikipedia.org/wiki/Zou_Jing_(engineer),3,2022,June,None,None,engineer,member of the Academy of Engineering,None,None,None,None,None,None,None,None,86.0,None,"China, People's Republic of",None,None,1.386294,0,0,0,0,0,0,0,0,0,0,0,0,0


<IPython.core.display.Javascript object>

In [4]:
# Checking a sample of the data
df.sample(5)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
69465,12,Stephen Reid,", 68, Canadian author and bank robber , heart failure and complications of pneumonia.",https://en.wikipedia.org/wiki/Stephen_Reid_(writer),12,2018,June,The Stopwatch Gang,None,bank robber,heart failure and complications of pneumonia,None,None,None,None,None,None,None,None,68.0,None,Canada,None,The Stopwatch Gang,2.564949,0,0,0,0,0,1,0,0,0,0,0,0,1
72926,28,John Raymond Smythies,", 96, British neuroscientist.",https://en.wikipedia.org/wiki/John_Raymond_Smythies,5,2019,January,None,None,neuroscientist,None,None,None,None,None,None,None,None,None,96.0,None,United Kingdom of Great Britain and Northern Ireland,None,None,1.791759,0,0,0,0,0,0,0,0,0,0,0,0,0
74697,9,Yuan Baohua,", 103, Chinese politician and academic administrator, Director of the State Economic Commission .",https://en.wikipedia.org/wiki/Yuan_Baohua,5,2019,May,", President of Renmin University",None,,Director of the State Economic Commission,None,None,None,None,None,None,None,None,103.0,None,"China, People's Republic of",None,"1981 1982, President of Renmin University 1985 1991",1.791759,0,0,0,1,0,0,0,0,1,0,0,0,2
61762,19,Rafael Kadyrov,", 47, Russian ice hockey referee, brain tumor.",https://en.wikipedia.org/wiki/Rafael_Kadyrov,5,2017,January,None,None,ice hockey referee,brain tumor,None,None,None,None,None,None,None,None,47.0,None,Russia,None,None,1.791759,0,0,0,0,0,0,0,0,0,0,0,0,0
66125,16,Gary the Goat,", 6, Australian comedic goat, euthanised.",https://en.wikipedia.org/wiki/Gary_the_Goat,14,2017,November,None,None,comedic goat,euthanised,None,None,None,None,None,None,None,None,6.0,None,Australia,None,None,2.708050,0,0,0,0,0,0,0,0,0,0,0,0,0


<IPython.core.display.Javascript object>

### Checking Data Types, Duplicates, and Null Values

In [5]:
# Checking data types and null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98059 entries, 0 to 98058
Data columns (total 38 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   day                        98059 non-null  object 
 1   name                       98059 non-null  object 
 2   info                       98059 non-null  object 
 3   link                       98059 non-null  object 
 4   num_references             98059 non-null  int64  
 5   year                       98059 non-null  int64  
 6   month                      98059 non-null  object 
 7   info_parenth               36661 non-null  object 
 8   info_1                     22 non-null     object 
 9   info_2                     98027 non-null  object 
 10  info_3                     48895 non-null  object 
 11  info_4                     10264 non-null  object 
 12  info_5                     1265 non-null   object 
 13  info_6                     181 non-null    obj

<IPython.core.display.Javascript object>

#### Observations:
- With our dataset loaded, we can pick up where we left off with extracting known_for values by rebuilding `known_for_dict`.

### Extracting `known_for` Continued

#### Finding `known_for` Roles in `info_2`

In [7]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [222]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [223]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [index for index in df.index if "educator" in df.loc[index, "info"]], "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [221]:
# # Code to check each specific value
# specific_roles_list.pop()

<IPython.core.display.Javascript object>

In [218]:
# # Example code to quick-screen values that may overlap categories
# df.loc[[index for index in df.index if "sex educator" in df.loc[index, "info"]]]

<IPython.core.display.Javascript object>

In [ ]:
# # Example code to quick-screen values that may overlap categories
# df.loc[
#     [
#         index
#         for index in df.index
#         if "outlaw country music singer songwriter" in df.loc[index, "info"]
#     ]
# ]

In [217]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "specialist in studies and educator"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [215]:
# Creating lists for each category
politics_govt_law = [
    "struggle veteran",
]

arts = []
sports = []
sciences = [
    "and wonen health",
]

business_farming = []
academia_humanities = [
    "educator and scholar",
    "educator and Māori language proponent",
    "and early childhood educator",
    "educator and professor",
    "scholar and educator",
    "educator and public schools superintendent",
    "AIDS educator and",
    "AIDS educator",
    "educator and director of the Advanced Placement Program",
    "educator and librarian",
    "adult educator",
    "university educator",
    "linguist and educator",
    "librarian and educator",
    "specialist in studies and educator",
    "Native educator",
    "educator and debate coach",
    "language linguist and educator",
    "Sanskrit scholar and educator",
    "educator and musicologist",
    "educator and anthropologist",
    "teacher and educator",
    "educator and college administrator",
    "Inuit educator",
    "educator and university administrator",
    "translator and educator",
    "and educator",
    "educator and",
    "educator",
]
law_enf_military_operator = []
spiritual = []
social = []
crime = []
event_record_other = []
other_species = []

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [216]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [224]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['academia_humanities'] ==1].sample(2)

CPU times: total: 16.3 s
Wall time: 16.3 s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
35978,12,Lowell H. Harrison,", 88, American historian.",https://en.wikipedia.org/wiki/Lowell_H._Harrison,3,2011,October,None,None,,None,None,None,None,None,None,None,None,None,88.0,None,United States of America,None,None,1.386294,0,0,0,1,0,0,0,0,0,0,0,0,1
66167,18,Peter Spufford,", 83, British historian.",https://en.wikipedia.org/wiki/Peter_Spufford,10,2017,November,None,None,,None,None,None,None,None,None,None,None,None,83.0,None,United Kingdom of Great Britain and Northern Ireland,None,None,2.397895,0,0,0,1,0,0,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [225]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 37846 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [227]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [429]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [428]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [index for index in df.index if "chemist" in df.loc[index, "info"]], "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [427]:
# # Code to check each specific value
# specific_roles_list.pop()

<IPython.core.display.Javascript object>

In [426]:
# # Example code to quick-screen values that may overlap categories
# df.loc[[index for index in df.index if "houngan" in df.loc[index, "info"]]]

<IPython.core.display.Javascript object>

In [425]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "chemist and astronaut candidate"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [423]:
# Creating lists for each category
politics_govt_law = []

arts = []
sports = []
sciences = [
    "theoretical chemist and Nobel Prize winner",
    "biochemist and molecular genetics pioneer",
    "native chemist",
    "biochemist and reproductive endocrinologist",
    "chemist and a winner of the Nobel Prize in Physics in",
    "chemist and Nobel Prize Laureate",
    "Nobel Prize winner in chemistry",
    "chemist and biologist",
    "biochemist and microbiologist",
    "nutritionist and biochemist",
    "neuroscience biochemist",
    "quantum chemist",
    "research chemist",
    "theoretical chemist and computer scientist",
    "research chemist and inventor",
    "chemist known for his work on the Manhattan Project",
    "biochemist and enzymologist",
    "scientist in the field of electrochemistry",
    "physical chemist at AT&T Bell Laboratories",
    "biochemist and Nobel Prize for Chemistry laureate",
    "industrial chemist best known for his work on polymers",
    "neurochemist and glycobiologist",
    "biochemist and virologist",
    "biochemist and recipient of the Nobel Prize in Physiology or Medicine",
    "physical and theoretical chemist",
    "chemist and mineralogist",
    "biochemist and cancer researcher",
    "biochemist and pharmacologist and recipient of the Nobel Prize in Physiology or Medicine",
    "nuclear chemist and recipient of the Nobel Prize in Chemistry",
    "chemist and co winner of Nobel Prize in Chemistry in",
    "botanist and chemist",
    "biochemist and protein crystallographer",
    "South wine chemist",
    "chemist and nuclear scientist",
    "oncologist and chemist",
    "chemist and science",
    "chemist and mass spectrometrist",
    "marine and freshwater chemist",
    "biochemist and molecular biologist",
    "protein chemist",
    "chemistry doctor",
    "Southern biochemist",
    "biochemist and developmental biologist",
    "biochemist and pharmacologist",
    "chemist and materials scientist",
    "biological chemist",
    "synthetic organic chemist",
    "atmospheric chemist",
    "chemist and researcher",
    "environmental chemist",
    "geochemist and planetary scientist",
    "physiologist and biochemist",
    "chemist and crystallographer",
    "scientist and agrochemist",
    "biochemist and physiologist",
    "biochemist and medical researcher",
    "textile chemist",
    "geologist and geochemist",
    "Congress chemist",
    "polymer chemist and inventor",
    "polymer chemist",
    "metallurgist and physical chemist",
    "chemist and astronaut candidate",
    "chemist and statistician",
    "cytologist and biochemist",
    "pharmacologist and biochemist",
    "natural product chemist",
    "chemist and Nobel laureate",
    "biochemist and Nobel Prize laureate",
    "biochemist and geneticist",
    "soil chemist and",
    "soil chemist",
    "biochemist and nutritionist",
    "pharmaceutical chemist",
    "Nobel Prize winning biochemist",
    "pharmacologist and chemist",
    "chemist and Nobel Prize laureate",
    "chemist and inventor",
    "biophysical chemist",
    "analytical chemist",
    "nuclear chemist",
    "electrochemist",
    "theoretical chemist",
    "inorganic chemist",
    "Nobel Prize winning chemist",
    "geochemist and",
    "geochemist",
    "physical chemist",
    "of organic chemistry and a",
    "of organic chemistry",
    "organic chemist and",
    "organic chemist",
    "of biochemistry",
    "and biochemist",
    "biochemist and",
    "biochemist",
    "of chemistry",
    "chemistry",
    "food chemist and",
    "chemist and",
    "and chemist",
    "chemist",
]

business_farming = []
academia_humanities = [
    "the first Rector Magnificus of the University of Twente",
]
law_enf_military_operator = []
spiritual = [
    "houngan",
]
social = []
crime = [
    "underground LSD chemist and",  # before sciences
]
event_record_other = []
other_species = []

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [424]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [430]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['sciences'] ==1].sample(2)

CPU times: total: 53.8 s
Wall time: 53.8 s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
68633,18,Karl Wolfgang Boer,", 92, German-born American physicist.",https://en.wikipedia.org/wiki/Karl_Wolfgang_Boer,10,2018,April,None,None,,None,None,None,None,None,None,None,None,None,92.0,None,Germany,United States of America,None,2.397895,1,0,0,0,0,0,0,0,0,0,0,0,1
2751,16,Brooke Benjamin,", 66, English mathematical physicist and mathematician.",https://en.wikipedia.org/wiki/Brooke_Benjamin,6,1995,August,None,None,,None,None,None,None,None,None,None,None,None,66.0,None,United Kingdom of Great Britain and Northern Ireland,None,None,1.945910,1,0,0,0,0,0,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [431]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 37258 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [433]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [472]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [436]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [index for index in df.index if "rugby union player" in df.loc[index, "info"]],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [469]:
# # Code to check each specific value
# specific_roles_list.pop()

<IPython.core.display.Javascript object>

In [470]:
# # Example code to quick-screen values that may overlap categories
# df.loc[
#     [
#         index
#         for index in df.index
#         if "rugby union player and administrator" in df.loc[index, "info"]
#     ]
# ]

<IPython.core.display.Javascript object>

In [ ]:
# # Example code to quick-screen values that may overlap categories
# df.loc[
#     [
#         index
#         for index in df.index
#         if "outlaw country music singer songwriter" in df.loc[index, "info"]
#     ]
# ]

In [471]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "rugby union player and administrator"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [473]:
# Creating lists for each category
politics_govt_law = []

arts = []
sports = [
    "rugby union player and World War II fighter",
    "rugby union player and TV",
    "former rugby union player for and Neath RFC",
    "All Blacks rugby union player",
    "rugby league and rugby union player",
    "rugby union player and president of the Rugby Union",
    "rugby union player and national team captain",
    "boxer and rugby union player",
    "Hall of Fame rugby union player and coach",
    "rugby union player and selector",
    "rugby union player and manager",
    "rugby union player and administrator",
    "rugby union player and coach",
    "international rugby union player",
    "rugby union player for",
    "rugby union player and",
    "and rugby union player",
    "rugby union player",
]
sciences = []

business_farming = []
academia_humanities = []
law_enf_military_operator = []
spiritual = []
social = []
crime = []
event_record_other = []
other_species = []

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [474]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [475]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['sports'] ==1].sample(2)

CPU times: total: 9.33 s
Wall time: 9.32 s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
93610,26,Bobby Kline,", 92, American baseball player .",https://en.wikipedia.org/wiki/Bobby_Kline,3,2021,October,Washington Senators,None,,None,None,None,None,None,None,None,None,None,92.0,None,United States of America,None,Washington Senators,1.386294,0,0,0,0,0,0,1,0,0,0,0,0,1
5663,24,Bill McArthur,", 78, American football player and coach.",https://en.wikipedia.org/wiki/Bill_McArthur_(American_football),3,1997,April,None,None,,None,None,None,None,None,None,None,None,None,78.0,None,United States of America,None,None,1.386294,0,0,0,0,0,0,1,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [476]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 36986 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [478]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [509]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [512]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [index for index in df.index if "rugby league player" in df.loc[index, "info"]],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [511]:
# # Code to check each specific value
# specific_roles_list.pop()

<IPython.core.display.Javascript object>

In [513]:
# # Example code to quick-screen values that may overlap categories
# df.loc[
#     [
#         index
#         for index in df.index
#         if "rugby league player and administrator" in df.loc[index, "info"]
#     ]
# ]

<IPython.core.display.Javascript object>

In [514]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "rugby league player involved in match fixing scandal"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [516]:
# Creating lists for each category
politics_govt_law = []

arts = []
sports = [
    "Hall of Fame rugby league player",
    "Papua New rugby league player",
    "rugby league player and captain",
    "rugby league player and international coach",
    "Hall of Fame rugby league player and national team captain",
    "football and rugby league player",
    "rugby league player for Wigan and Great",
    "rugby league player for Great and Hull KR",
    "rugby league player for Great",
    "rugby league player and referee",
    "rugby league player and administrator",
    "rugby union and rugby league player",
    "rugby league player and coach",
    "and rugby league player",
    "rugby league player and",
    "rugby league player",
]
sciences = []

business_farming = []
academia_humanities = []
law_enf_military_operator = []
spiritual = []
social = []
crime = ["involved in match fixing scandal"]
event_record_other = []
other_species = []

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [517]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [518]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['sports'] ==1].sample(2)

CPU times: total: 8.88 s
Wall time: 8.89 s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
37887,25,Tony Gordon,", 63, New Zealand rugby league player and coach.",https://en.wikipedia.org/wiki/Tony_Gordon_(rugby),17,2012,March,None,None,,None,None,None,None,None,None,None,None,None,63.0,None,New Zealand,None,None,2.890372,0,0,0,0,0,0,1,0,0,0,0,0,1
40166,6,Irene DeLaby,", 90, American baseball player.",https://en.wikipedia.org/wiki/Irene_DeLaby,7,2012,October,None,None,,None,None,None,None,None,None,None,None,None,90.0,None,United States of America,None,None,2.079442,0,0,0,0,0,0,1,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [519]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 36706 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [521]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [589]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [524]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [index for index in df.index if "sociologist" in df.loc[index, "info"]],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [590]:
# # Code to check each specific value
# specific_roles_list.pop()

<IPython.core.display.Javascript object>

In [591]:
# # Example code to quick-screen values that may overlap categories
# df.loc[
#     [index for index in df.index if "historical sociologist" in df.loc[index, "info"]]
# ]

<IPython.core.display.Javascript object>

In [592]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "sociologist and critic of systems analysis"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [593]:
# Creating lists for each category
politics_govt_law = []

arts = []
sports = [
    "caver",
]
sciences = [
    "sociologist and communication scientist",
    "sociologist and social justice",
    "sociologist and social",
    "sociologist and psychiatrist",
    "sociologist and critic of systems analysis",
    "sociologist of science",
    "criminologist and sociologist",
    "gerontologist and sociologist",
    "urban sociologist",
    "sociologist and patients' rights advocate",
    "sociologist and statistician",
    "environmental sociologist",
    "sociologist and demographer",
    "sociologist and criminologist",
    "sociologist from",
    "sociologist and",
    "sociologist",
]

business_farming = []
academia_humanities = []
law_enf_military_operator = []
spiritual = []
social = []
crime = []
event_record_other = []
other_species = []

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [594]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [595]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['sciences'] ==1].sample(2)

CPU times: total: 9.47 s
Wall time: 9.46 s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
35135,8,William R. Corliss,", 84, American physicist and writer.",https://en.wikipedia.org/wiki/William_R._Corliss,12,2011,July,None,None,,None,None,None,None,None,None,None,None,None,84.0,None,United States of America,None,None,2.564949,1,0,0,0,0,1,0,0,0,0,0,0,2
50027,30,Martin Lewis Perl,", 87, American physicist, discovered the tau particle, Nobel Prize laureate in Physics , heart attack.",https://en.wikipedia.org/wiki/Martin_Lewis_Perl,13,2014,September,,None,,discovered the tau particle,Nobel Prize laureate in Physics,heart attack,None,None,None,None,None,None,87.0,None,United States of America,None,1995,2.639057,1,0,0,0,0,0,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [596]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 36474 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [598]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [764]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [763]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [index for index in df.index if "psychologist" in df.loc[index, "info"]],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [762]:
# # Code to check each specific value
# specific_roles_list.pop()

<IPython.core.display.Javascript object>

In [761]:
# # Example code to quick-screen values that may overlap categories
# df.loc[
#     [
#         index
#         for index in df.index
#         if "and pastoral psychologist" in df.loc[index, "info"]
#     ]
# ]

<IPython.core.display.Javascript object>

In [ ]:
# # Example code to quick-screen values that may overlap categories
# df.loc[
#     [
#         index
#         for index in df.index
#         if "outlaw country music singer songwriter" in df.loc[index, "info"]
#     ]
# ]

In [760]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "educational psychologist"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [758]:
# Creating lists for each category
politics_govt_law = ["children ombudsman"]

arts = []
sports = []
sciences = [
    "clinical psychologist and researcher",
    "psychologist renowned for his critical studies of hypnosis",
    "psychologist and expert in the work of Alfred Adler",
    "psychologist and sex researcher at Johns Hopkins University",
    "psychologist and methodologist",
    "psychologist known for his pioneering work in autism treatment",
    "psychologist and women reproductive health",
    "traffic psychologist",
    "psychologist and researcher",
    "psychologist and former head of the Psychological Association",
    "correctional psychologist and criminologist",
    "behavioral geneticist and psychologist",
    "psychologist and psychoanalyst",
    "psychologist at University",
    "existential psychologist",
    "psychologist who specialized in developmental psychology",
    "psychologist and a pioneer in experimental psychology",
    "child development psychologist",
    "development psychologist",
    "psychologist and statistician",
    "pediatric psychologist",
    "psychologist from the Antilles",
    "social psychologist and sexologist",
    "psychologist and sleep researcher",
    "psychologist and computer science researcher",
    "social psychologist and criminologist",
    "parapsychologist and ufologist",
    "mathematical psychologist and",
    "mathematical psychologist",
    "engineering psychologist",
    "psychiatrist and parapsychologist",
    "psychologist and anti divorce",
    "evolutionary psychologist",
    "sexologist and psychologist",
    "systems psychologist",
    "environmental psychologist",
    "psychologist and self help",
    "and pastoral psychologist",
    "organizational psychologist",
    "neuroscientist and psychologist",
    "child psychologist and",
    "child psychologist",
    "psychologist and parapsychologist",
    "experimental psychologist",
    "psychologist and neuroscientist",
    "parapsychologist and",
    "and parapsychologist",
    "parapsychologist",
    "neuropsychologist",
    "cognitive psychologist",
    "developmental psychologist",
    "clinical psychologist",
    "social psychologist",
    "behavioral psychologist",
    "Jungian psychologist and",
    "and psychologist",
    "psychologist and",
    "psychologist",
]

business_farming = []
academia_humanities = []
law_enf_military_operator = []
spiritual = []
social = []
crime = []
event_record_other = []
other_species = []

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [759]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [ ]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['arts'] ==1].sample(2)

#### Checking the Number of Rows without a First Category

In [ ]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

In [597]:
print("dunzo!")

# Sound notification when cell executes
chime.success()

dunzo!


<IPython.core.display.Javascript object>

#### Finding `known_for` Roles in `info_2`

In [ ]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

In [ ]:
# Code to check each value
roles_list.pop()

In [ ]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [index for index in df.index if "writer" in df.loc[index, "info"]], "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

In [ ]:
# # Code to check each specific value
# specific_roles_list.pop()

In [ ]:
# # Example code to quick-screen values that may overlap categories
# df.loc[[index for index in df.index if "and science writer" in df.loc[index, "info"]]]

In [ ]:
# # Example code to quick-screen values that may overlap categories
# df.loc[
#     [
#         index
#         for index in df.index
#         if "outlaw country music singer songwriter" in df.loc[index, "info"]
#     ]
# ]

In [ ]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "outlaw country music singer songwriter"]

#### Creating Lists for Each `known_for` Category

In [ ]:
# Creating lists for each category
politics_govt_law = []

arts = []
sports = [


]
sciences = []

business_farming = []
academia_humanities = []
law_enf_military_operator = []
spiritual = []
social = []
crime = []
event_record_other = []
other_species = []

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [ ]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

#### Extracting Category from `info_2`

In [ ]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['arts'] ==1].sample(2)

#### Checking the Number of Rows without a First Category

In [ ]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Observations:
- It is time to export our dataframe and start a new notebook.

### Exporting Dataset to SQLite Database [wp_life_expect_clean8.db]()

In [ ]:
# # Exporting dataframe

# # Saving dataset in a SQLite database
# conn = sql.connect("wp_life_expect_clean8.db")
# df.to_sql("wp_life_expect_clean8", conn, index=False)

# # Chime notification when cell executes
# chime.success()

# [Proceed to Data Cleaning Part 9 ]()